In [ ]:
import torch
import collections
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
import sys
sys.path.insert(1, '/home/buehlern/Documents/Masterarbeit/models')
from src.data.mri_datamodule import MRIDataModule
from src.models.vit_mae_module import VisionTransformerMAE

/home/buehlern/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load model checkpoint
mae_checkpoint = "/home/buehlern/Documents/Masterarbeit/models/logs/train/multiruns/2024-07-29_12-12-24/0/checkpoints/epoch_009.ckpt"
checkpoint = torch.load(mae_checkpoint)
state_dict = checkpoint['state_dict']

/tmp/ipykernel_1513142/2628517626.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(mae_checkpoint)
/home/buehlern/.local/lib/python3.10/site-packa

In [4]:
state_dict.keys()

odict_keys(['net._orig_mod.vit.embeddings.cls_token', 'net._orig_mod.vit.embeddings.position_embeddings', 'net._orig_mod.vit.embeddings.patch_embeddings.projection.weight', 'net._orig_mod.vit.embeddings.patch_embeddings.projection.bias', 'net._orig_mod.vit.encoder.layer.0.attention.attention.query.weight', 'net._orig_mod.vit.encoder.layer.0.attention.attention.query.bias', 'net._orig_mod.vit.encoder.layer.0.attention.attention.key.weight', 'net._orig_mod.vit.encoder.layer.0.attention.attention.key.bias', 'net._orig_mod.vit.encoder.layer.0.attention.attention.value.weight', 'net._orig_mod.vit.encoder.layer.0.attention.attention.value.bias', 'net._orig_mod.vit.encoder.layer.0.attention.output.dense.weight', 'net._orig_mod.vit.encoder.layer.0.attention.output.dense.bias', 'net._orig_mod.vit.encoder.layer.0.intermediate.dense.weight', 'net._orig_mod.vit.encoder.layer.0.intermediate.dense.bias', 'net._orig_mod.vit.encoder.layer.0.output.dense.weight', 'net._orig_mod.vit.encoder.layer.0.outp

In [5]:
def remap_keys(state_dict, unwanted_prefix, new_prefix):
    new_state_dict = {}
    for key, value in state_dict.items():
        if key.startswith(unwanted_prefix):
            new_key = key.replace(unwanted_prefix, new_prefix)
        else:
            new_key = key
        new_state_dict[new_key] = value
    return collections.OrderedDict(new_state_dict)

In [6]:
new_state_dict = remap_keys(state_dict, 'net._orig_mod.', 'net.')

In [7]:
new_state_dict.keys()

odict_keys(['net.vit.embeddings.cls_token', 'net.vit.embeddings.position_embeddings', 'net.vit.embeddings.patch_embeddings.projection.weight', 'net.vit.embeddings.patch_embeddings.projection.bias', 'net.vit.encoder.layer.0.attention.attention.query.weight', 'net.vit.encoder.layer.0.attention.attention.query.bias', 'net.vit.encoder.layer.0.attention.attention.key.weight', 'net.vit.encoder.layer.0.attention.attention.key.bias', 'net.vit.encoder.layer.0.attention.attention.value.weight', 'net.vit.encoder.layer.0.attention.attention.value.bias', 'net.vit.encoder.layer.0.attention.output.dense.weight', 'net.vit.encoder.layer.0.attention.output.dense.bias', 'net.vit.encoder.layer.0.intermediate.dense.weight', 'net.vit.encoder.layer.0.intermediate.dense.bias', 'net.vit.encoder.layer.0.output.dense.weight', 'net.vit.encoder.layer.0.output.dense.bias', 'net.vit.encoder.layer.0.layernorm_before.weight', 'net.vit.encoder.layer.0.layernorm_before.bias', 'net.vit.encoder.layer.0.layernorm_after.wei

In [8]:
mae = VisionTransformerMAE(image_size = 3072, patch_size = 48)
mae.load_state_dict(new_state_dict)

<All keys matched successfully>

In [9]:
mae.eval()

VisionTransformerMAE(
  (net): ViTMAEForPreTraining(
    (vit): ViTMAEModel(
      (embeddings): ViTMAEEmbeddings(
        (patch_embeddings): ViTMAEPatchEmbeddings(
          (projection): Conv2d(1, 768, kernel_size=(48, 48), stride=(48, 48))
        )
      )
      (encoder): ViTMAEEncoder(
        (layer): ModuleList(
          (0-23): 24 x ViTMAELayer(
            (attention): ViTMAESdpaAttention(
              (attention): ViTMAESdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): ViTMAESelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
            (intermediate): ViT

In [10]:
# Disable masking
#mae.net.config.mask_ratio = 0

In [11]:
# Load the DataModule
mri_datamodule = MRIDataModule(image_size = 3072, square = True, output_channels = 1, cache = False, fix_inverted = True)

initializing MRIDatasetBase ...
reading /home/buehlern/Documents/Masterarbeit/data/clean_df_slim.pkl file ...
PATH /home/buehlern/Documents/Masterarbeit/data/BodyPartExamined_mappings_mergemore.json
/home/buehlern/Documents/Masterarbeit/data/cache-full/df_labelcomparison.pkl does not exit --> no items excluded by it
MRIDatasetBase(len=639877) initialized

initializing MRIDataset(mode=train) ...
MRIDataset(mode=train, len=516402) initialized

initializing MRIDataset(mode=val) ...
MRIDataset(mode=val, len=27518) initialized

initializing MRIDataset(mode=test) ...
WARN: including test data
MRIDataset(mode=test, len=95957) initialized


In [13]:
def show_image(image, title=''):
    # image is [H, W, 1]
    assert image.shape[2] == 1
    plt.imshow(image, cmap=plt.cm.bone)
    plt.title(title, fontsize=16)
    plt.axis('off')
    return

In [54]:
def visualize(pixel_values, model, imgname=None):
    # forward pass
    outputs = model(pixel_values)
    y = model.unpatchify(outputs.logits)
    y = torch.einsum('nchw->nhwc', y).detach().cpu()
    
    # visualize the mask
    mask = outputs.mask.detach()
    mask = mask.unsqueeze(-1).repeat(1, 1, model.config.patch_size**2 *1)  # (N, H*W, p*p*1)
    mask = model.unpatchify(mask)  # 1 is removing, 0 is keeping
    mask = torch.einsum('nchw->nhwc', mask).detach().cpu()
    
    x = torch.einsum('nchw->nhwc', pixel_values)

    # masked image
    im_masked = x * (1 - mask)

    # MAE reconstruction pasted with visible patches
    im_paste = x * (1 - mask) + y * mask

    # make the plt figure larger
    plt.rcParams['figure.figsize'] = [24, 10]

    plt.subplot(1, 4, 1)
    show_image(x[0], "original")

    plt.subplot(1, 4, 2)
    show_image(im_masked[0], "masked")

    plt.subplot(1, 4, 3)
    show_image(y[0], f"reconstruction (loss: {outputs.loss.item():.4f})")

    plt.subplot(1, 4, 4)
    show_image(im_paste[0], "reconstruction + visible")

    if imgname is not None:
        plt.savefig('/home/buehlern/Documents/Masterarbeit/notebooks/Data Exploration Graphics/Model Eval/ViT MAE/' + str(imgname) + '.png')
    plt.show()

In [ ]:
dl_iter = iter(mri_datamodule.data_val)
shuf_dl_iter = iter(random.sample(list(mri_datamodule.data_val), len(mri_datamodule.data_val)))

In [ ]:
for i in range(50):
    item = next(shuf_dl_iter)
    image = item[0]
    batch = image.unsqueeze(0)
    visualize(batch, mae.net, imgname=i)